In [ ]:
import os
import numpy as np
from PIL import Image
import os

dataset_path = "train/images/"
output_folder = "train/augmented_images/"

AUG_PER_IMAGE = 3
LSB_BITS = 4

In [ ]:
def load_images_from_folder(folder):
    images = []
    names = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, filename)
            img = Image.open(path).convert('RGB')
            images.append(img)
            names.append(os.path.splitext(filename)[0])  # filename senza estensione
    return images, names

images, names = load_images_from_folder(dataset_path)
print(f"Loaded {len(images)} images from {dataset_path}")

In [38]:
def apply_lsb_noise(images, bits=4, n_aug=3):
    noisy_images = []
    for img in images:
        img_array = np.array(img)
        print("Image pixels:", img_array[0, 0])
        for i in range(n_aug):
            # Change the last 'bits' bits of each pixel
            # Create a mask to clear the last 'bits' bits:
            # 0xFF is 11111111 in binary
            # 2^bits - 1 (e.g., for bits=4, this is 00001111)
            mask = 0xFF ^ ((2**bits) - 1)
            noise = np.random.randint(0, 2**bits, img_array.shape, dtype=np.uint8)
            noisy_img = (img_array & mask) | noise
            print(f"Augmented image {i+1} pixels:", noisy_img[0, 0])
            noisy_images.append(Image.fromarray(noisy_img))
    return noisy_images

def save_augmented_images(names, noisy_images, output_folder, n_aug=3):
    os.makedirs(output_folder, exist_ok=True)
    for name in names:
        for i in range(n_aug):
            out_name = f"{name}_aug_{i+1}.png"
            aug_img = noisy_images.pop(0)
            aug_img.save(os.path.join(output_folder, out_name))

In [ ]:
noisy_images = apply_lsb_noise(images, LSB_BITS)
print(f"Generated {len(noisy_images)} noisy images with LSB noise.")
save_augmented_images(names, noisy_images, output_folder, AUG_PER_IMAGE)

In [40]:
def pixel_difference(img1, img2):
    print("Size of images:", img1.size, img2.size)
    arr1 = np.array(img1).astype(np.int16)
    arr2 = np.array(img2).astype(np.int16)
    diff = np.abs(arr1 - arr2)

    print("Pixel modificati:", np.sum(diff > 0))
    print("Massima differenza:", np.max(diff))
    print("Media differenza:", np.mean(diff))
    print("Somma totale differenza:", np.sum(diff))

aug_image_path = "train/augmented_images/05-15_00078_P0030852_aug_1.png"
image_path = "train/images/05-15_00078_P0030852.png"
image = Image.open(image_path).convert('RGB')

noise_image = apply_lsb_noise([image], LSB_BITS, 1)[0]
aug_image = Image.open(aug_image_path).convert('RGB')
pixel_difference(image, aug_image)

Image pixels: [110  73  39]
Augmented image 1 pixels: [98 64 36]
Size of images: (1024, 1024) (1024, 1024)
Pixel modificati: 2949665
Massima differenza: 15
Media differenza: 5.309468905131022
Somma totale differenza: 16702145
